In [4]:
import pandas as pd
import os
import numpy as np

In [3]:
os.chdir(r"G:\2023-纵向\停车分配\save_data_0923\需求11供给11")
os.getcwd()

'G:\\2023-纵向\\停车分配\\save_data_0923\\需求11供给11'

In [15]:
park_fee = 3 / 2  # 半个小时的费用
charge_fee = [0, 0.7, 0.5]  # 每分钟的价格
reserved_fee = 1

def get_req_info(foldername):
    folder_path = "G://2023-纵向//停车分配//需求分布//demand0607//"
    file_path = folder_path + foldername+ ".csv"
    req_info =  pd.read_csv(file_path)
    req_info['revenue'] = [(np.floor(req_info['activity_t'].iloc[i] / 15) * park_fee + (
        req_info['activity_t'].iloc[i] * (charge_fee[req_info['new_label'].iloc[i]])) + reserved_fee) for i in
                               range(len(req_info))]
    req_info['request_interval'] = (req_info['request_t'] - min(req_info['request_t'])) // 15
    return req_info


def revise_assign_info(base_dir, skip_folders=None):
    if skip_folders is None:
        skip_folders = []

    # 遍历所有的子文件夹
    for root, dirs, files in os.walk(base_dir):
        # 跳过指定的文件夹
        dirs[:] = [d for d in dirs if d not in skip_folders]

        # 检查是否在 result_info 目录下
        if 'assign_info' in root:
            folder_name = os.path.basename(os.path.dirname(root))  # 例如 '50-1', '75-1'

            for file in files:
                if file.split('_')[0] in ['rdp','so']:
                    file_path = base_dir+"//"+folder_name+"//assign_info//"+file
                    assign_info = pd.read_csv(file_path)
                    req_info = get_req_info(folder_name)
                    dpr_assign_filter = assign_info.loc[assign_info.groupby('req_id')['assign_t'].idxmax()]
                    # dpr_assign_filter_merge = pd.merge(dpr_assign_filter, req_info[['req_id', 'revenue','label']], on='req_id', how='left')
                    dpr_assign_filter_merge = pd.merge(dpr_assign_filter, req_info[['req_id','request_interval']], on='req_id', how='left')
                    dpr_assign_filter_merge.to_csv(file_path,index=False)
# 设置要跳过的文件夹
skip_folders = [str(i)+'-1'for i in range(525,1025,25)]
skip_folders.extend(['25-1','test_fbfs'])

# 正确的 base_dir 路径设置
base_dir = 'G://2023-纵向//停车分配//save_data_0923//需求11供给11'


In [16]:
revise_assign_info(base_dir)